# Supp. Mat. - moving threshold

This follows the Naive Bayes module, and expands it to have a little more depth to it with priors and a probabilistic output; most of the setup code is shared with the NBC notebook, but duplication is fine as it makes the two notebooks independent.


In [ ]:
_code_path = joinpath(@__DIR__, "..", "lib")
include(joinpath(_code_path, "pkg.jl"))
include(joinpath(_code_path, "confusiontable.jl"))
include(joinpath(_code_path, "nbc.jl"))
include(joinpath(_code_path, "splitters.jl"))

We load the data


In [ ]:
_ptm_path = joinpath(@__DIR__, "..", "ptm")
modelpath = joinpath(_ptm_path, "reindeer-varselected.jld")
ptm = JLD.load(modelpath)
trainlabels, trainfeatures = ptm["training"]
testlabels, testfeatures = ptm["testing"]

MC CV


In [ ]:
thresholds = collect(LinRange(0.0, 1.0, 300))
k = 10
conftrain = zeros(ConfusionMatrix, (k, length(thresholds)))
confvalid = zeros(ConfusionMatrix, (k, length(thresholds)))

folds = kfold(trainlabels, trainfeatures; k=k, permute=true)

for f in axes(folds, 1)
    t_idx, v_idx = folds[f]
    predictor = naivebayes(trainlabels[t_idx], trainfeatures[t_idx,:])
    # Probability predictions
    prediction = vec(mapslices(predictor, trainfeatures[t_idx,:], dims=2))
    validation = vec(mapslices(predictor, trainfeatures[v_idx,:], dims=2))
    # Thresholds
    for (i,t) in enumerate(thresholds)
        conftrain[f,i] = ConfusionMatrix(prediction, trainlabels[t_idx], t)
        confvalid[f,i] = ConfusionMatrix(validation, trainlabels[v_idx], t)
    end
end

how good are our folds


In [ ]:
#| label: fig-moving-roc-pr
#| fig-cap: roc and pr curves
raw_tpr = tpr.(confvalid)
raw_fpr = fpr.(confvalid)
raw_ppv = ppv.(confvalid)
f = Figure(; resolution=(500, 300))
roc = Axis(f[1,1]; xlabel="False Positive Rate", ylabel="True Positive Rate")
pr = Axis(f[1,2]; xlabel="True Positive Rate", ylabel="Positive Predictive Value")
scatter!(roc, [0.0], [1.0], color=:lightgrey, markersize=80)
scatter!(pr, [1.0], [1.0], color=:lightgrey, markersize=80)
for i in axes(confvalid, 1)
    lines!(roc, raw_fpr[i,:], raw_tpr[i,:], color=(:black, 0.2))
    lines!(pr, raw_tpr[i,:], raw_ppv[i,:], color=(:black, 0.2))
end
for ax in [roc, pr]
    xlims!(ax, (0., 1.0))
    ylims!(ax, (0., 1.0))
    tightlimits!(ax)
end
hlines!(pr, [0.5], color=:black, linestyle=:dash)
ablines!(roc, [0.0], [1.0], color=:black, linestyle=:dash)
current_figure()

pretty good -- now we need to pick an optimal value for the threshold


In [ ]:
#| label: fig-moving-thresholding
#| fig-cap: performance tuning curve
raw_mcc = mcc.(confvalid)
raw_mcc[findall(isnan, raw_mcc)] .= 0.0
raw_tss = trueskill.(confvalid)
raw_tss[findall(isnan, raw_tss)] .= 0.0

f = Figure(; resolution=(500, 300))
axmcc = Axis(f[1,1]; xlabel="Threshold", ylabel="MCC")
axtss = Axis(f[1,2]; xlabel="Threshold", ylabel="True Skill Statistic")
ylims!(axmcc, low=0.0, high=1.0)
xlims!(axmcc, low=0.0, high=1.0)
xlims!(axtss, low=0.0, high=1.0)
ylims!(axtss, low=0.0, high=1.0)
for i in axes(confvalid, 1)
    lines!(axmcc, thresholds, raw_mcc[i,:], color=(:black, 0.2))
    lines!(axtss, thresholds, raw_tss[i,:], color=(:black, 0.2))
end

lines!(axmcc, thresholds, vec(mean(raw_mcc; dims=1)), color=:black, linewidth=3, linestyle=:dash)
lines!(axtss, thresholds, vec(mean(raw_tss; dims=1)), color=:black, linewidth=3, linestyle=:dash)

current_figure()

pick the threshold


In [ ]:
mcc_perf = vec(mean(raw_mcc; dims=1))
τ = thresholds[last(findmax(mcc_perf))]
@info τ

train the final model


In [ ]:
predictor = naivebayes(trainlabels, trainfeatures)

and test on the testing data


In [ ]:
#| label: tbl-moving-confusion
#| tbl-cap: confusion table after tuning the value of $\tau$ etc etc
prediction = vec(mapslices(predictor, testfeatures, dims=2))
conf = ConfusionMatrix(prediction, testlabels, τ)
out = pretty_table(String, Matrix(conf), tf = tf_html_simple, formatters = ft_nomissing, show_subheader = false, show_header = false)
display("text/html", out)

but we can add an extra step! optimize the prior and the threshold at the same time using grid search using two loops, first the prior and then the thresholding  -- this is rather long to run, so we actually do the search only for a relevant range of parameters


In [ ]:
thresholds = collect(LinRange(0.0, 1.0, 100))
priors = collect(LinRange(0.5, 0.65, 50))
scores = zeros(Float64, (length(priors), length(thresholds)))

k = 10
folds = kfold(trainlabels, trainfeatures; k=k, permute=true)
conftrain = zeros(ConfusionMatrix, (k, length(thresholds)))
confvalid = zeros(ConfusionMatrix, (k, length(thresholds)))

Threads.@threads for i in axes(priors, 1)
    @info priors[i]
    for f in axes(folds, 1)
        t_idx, v_idx = folds[f]
        predictor = naivebayes(trainlabels[t_idx], trainfeatures[t_idx,:]; presence=priors[i])
        # Probability predictions
        prediction = vec(mapslices(predictor, trainfeatures[t_idx,:], dims=2))
        validation = vec(mapslices(predictor, trainfeatures[v_idx,:], dims=2))
        # Thresholds
        for (ti,t) in enumerate(thresholds)
            conftrain[f,ti] = ConfusionMatrix(prediction, trainlabels[t_idx], t)
            confvalid[f,ti] = ConfusionMatrix(validation, trainlabels[v_idx], t)
        end
    end
    scores[i,:] .= vec(mean(mcc.(confvalid), dims=1))
    @info findmax(scores)
end

look at the grid at the end of the tuning process


In [ ]:
heatmap(scores)

we get the best parameters


In [ ]:
best_pos = last(findmax(scores))
prior = priors[best_pos[1]]
τ = thresholds[best_pos[2]]
@info prior, τ

now we run the prediction on the model with both parameters tuned


In [ ]:
predictor = naivebayes(trainlabels, trainfeatures; presence=prior)

output the table


In [ ]:
#| label: tbl-moving-confusion-grid
#| tbl-cap: confusion table after tuning the value of $\tau$ and the prior etc etc
prediction = vec(mapslices(predictor, testfeatures, dims=2))
conf = ConfusionMatrix(prediction, testlabels, τ)
out = pretty_table(String, Matrix(conf), tf = tf_html_simple, formatters = ft_nomissing, show_subheader = false, show_header = false)
display("text/html", out)

look at probas - this explains why the thresholding curve was essentially flat, the model is *really* good at picking 0/1 probabilities


In [ ]:
#| label: fig-moving-explanation
#| fig-cap: probas etc etc
f = Figure()
gl = f[1,1] = GridLayout()

axp = Axis(gl[1,1])
hist!(axp, prediction, normalization=:probability, bins=10, color=:lightgrey)
xlims!(axp, low=0.0, high=1.0)
ylims!(axp, low=0.0)
hidespines!(axp)
hidedecorations!(axp)

axr = Axis(gl[2,1]; xlabel="Predicted probability for the presence class")
colpos = Makie.wong_colors()[6]
cl = [(p ? colpos : :grey) for p in ((prediction .>= τ).==testlabels)]
rainclouds!(axr, testlabels, prediction, plot_boxplots=false, clouds=nothing, side_nudge=0.0, orientation=:horizontal, jitter_width=0.5, markersize=8, color=cl)
xlims!(axr, low=0.0, high=1.0)
ylims!(axr, low=-0.5, high=1.5)

vlines!(axr, τ, color=:black, linestyle=:dash)

rowgap!(gl, 0)
rowsize!(gl, 1, Relative(1/7))

current_figure()

In [ ]:
@info tpr(conf)
@info tnr(conf)
@info fpr(conf)
@info fnr(conf)
@info trueskill(conf)
@info mcc(conf)

get the data


In [ ]:
using SpeciesDistributionToolkit
_layer_path = joinpath(@__DIR__, "..", "data", "general", "rangifer-layers.tiff")
bio = [SpeciesDistributionToolkit._read_geotiff(_layer_path, SimpleSDMResponse; bandnumber=i) for i in ptm["varidx"]]

make the predictions for every pixel and get the entropy


In [ ]:
pr = convert(Float64, similar(first(bio)))
entropy = similar(pr)
Threads.@threads for k in keys(pr)
    x = [b[k] for b in bio]
    pr[k] = predictor(x)
    p = [pr[k], 1.0 - pr[k]]
    entropy[k] = -sum(p .* log2.(p))
end

show the results on a map


In [ ]:
#| label: fig-moving-rangemap
#| fig-cap: Range map of *Rangifer tarandus tarandus* as ...
f = Figure(; resolution=(500, 300))

gl = f[1,1] = GridLayout()


axpt = Axis(gl[1,1]; ylabel="Latitude", xgridvisible=false, ygridvisible=false)
heatmap!(axpt, pr .> τ, colormap=[:lightgrey, :grey])
#scatter!(axpt, presences.longitude, presences.latitude, color=presences.presence, markersize=1, colormap=[:grey, :black])

axpr = Axis(gl[1,2]; xlabel="Longitude", ylabel="", xgridvisible=false, ygridvisible=false)
heatmap!(axpr, pr, colormap=[:lightgrey, :darkgreen])

axen = Axis(gl[2,1]; xlabel="Longitude", ylabel="Latitude", xgridvisible=false, ygridvisible=false)
heatmap!(axen, entropy, colormap=[:lightgrey, :darkred])

hidexdecorations!(axpt)
hideydecorations!(axpr)
hidexdecorations!(axpr)
hideydecorations!(axen)

rowgap!(gl, 0)
colgap!(gl, 0)

current_figure()

Finally we _save_ the model to use it in the next modules


In [ ]:
_ptm_path = joinpath(@__DIR__, "..", "ptm")
modelpath = joinpath(_ptm_path, "reindeer-tuned.jld")
JLD.save(
    modelpath,
    "threshold", τ,
    "prior", prior,
    "testing", (testlabels, testfeatures),
    "varidx", ptm["varidx"],
    "C", conf,
    "training", (trainlabels, trainfeatures)
)